# Assignment Number 3: Preprocessing Data for Record Linkage

-----

Name: Shiming Xiong

-----

---

Net id: sx368

---

The goal of this assignment is to clean data of two files to harmonize them so we can use them later to do a record linkage. The files we will be using as a baseline for this assignment is the inventor.csv that is in our project folder in the Data subfolder. This file contains names of inventors and location. Our goal is to find out if any of these inventors received a grant in 2017. Thus we need to merge the inventor file, to the file that contains information on grant receipt in 2017 (is located in our Data folder under projedcts). The merge then would happen on names.

However, merging the data as is on name won't be sucessful, as each dataset (inventors and grants) contain names, but they are formatted differently. The linkage will only work if all the variable we are merging on look exactly the same.

This assignment will have you prepare the grants data in a way that it matches the format of the inventor file. Please follow the instructions below. 

## List of Individual Tasks

1. Load the inventor file and check with a head statement how the dataset looks like. Describe what you see, e.g. names are upper case/ lower case, first and last name are seperate vs in one files. Does the location information have errors, e.g. is California always referenced as CA or also California or Cali. Tell us anything that you notice. (1.5 points)

2. Load the file that has the information on grants. Keep only the names of the PI and co-PIs, the zipcode, state, and city of the organization. Then check these fields and see how they are recorded in this dataset. How do they differ from the inventor file? (1.5 points)

3. Clean the names in the grant data. You want to bring them into the same format as in the inventor data. You can use string functions like in the class notebook to extract content from a field. (2 points)

4. We also can link not only by name but location. Make sure that both datasets only contain valid entries for U.S. States. This might be helpful: https://sceptermarketing.com/comma-separated-lists-of-us-states-abbreviations-select-options-etc/. Then you can also look at the country variable. (2 points)

5. Prepare zipcode of the grants file. Generate a variable that only contains the 5 digit zipcode in the U.S. You can use regex commands for this. A good website to test the regex commands is this one. https://regexr.com/. In order to use regex you need to import the package (import re). Here is a nice website that provides you with more details on how to exctract information from strings with regex https://chrisalbon.com/python/data_wrangling/pandas_regex_to_create_columns/ (2 points)

6. Save the final grants dataset as csv. It should contain following columns: name_first, name_last, zip, state, city, country. Make sure everything is lower case. (1 point) 

## Extra Points

If you want to engage in a little python challenge you can do this to get extra points. You can apply the extra points at the end of class and use ot to make up for missed points in other assignments. This is voluntary and you don't need to do this additional exercise. But give it a try. It is fun!! You can get 2 extra points.

1. Load the grant data from 2017 again (in the project folder) and only keep the variable organization_name. If you look at the names you can see that these are not consistent at all. Give it a try to clean them. Generate a new variable that contains the name of the organization, but is consistent over the entire dataset. 

In [1]:
# general use 
%pylab inline
import datetime
import numpy as np
import os
import six
import warnings
import matplotlib.pyplot as plt
import re

# pandas
from __future__ import print_function
import pandas as pd
import scipy
import sklearn

# record linkage package
import recordlinkage as rl
from recordlinkage.preprocessing import clean, phonenumbers, phonetic

# CSV file reading
import csv

# sqlalchemy an psycopg2 are sql connection packages
from sqlalchemy import create_engine


print( "Imports loaded at " + str( datetime.datetime.now() ) )

Populating the interactive namespace from numpy and matplotlib
Imports loaded at 2019-03-15 07:35:11.737227


In [2]:
data_dir = "~/projects/wagner_ada_2019/shared/Data"
print( "Data directory for Project Data = " + data_dir )

Data directory for Project Data = ~/projects/wagner_ada_2019/shared/Data


## Question 1

In [3]:
inventor=pd.read_csv("~/projects/wagner_ada_2019/shared/Data/inventor.csv",error_bad_lines=False, encoding='utf8')

In [4]:
inventor.head()

,Unnamed: 0,inventor_id,city,state,country,latitude,longitude,state_fips,county_fips,name_first,name_last
0,0,4305013-1,Kaiserslautern,NaN,DE,49.4123,7.69879,NaN,NaN,Paul Walter,Baier
1,1,4305013-1,Kaiserslautern,NaN,DE,49.4123,7.69879,NaN,NaN,Paul Walter,Baier
2,2,4305013-1,Kaiserslautern,NaN,DE,49.4123,7.69879,NaN,NaN,Paul Walter,Baier
3,3,4305013-1,Kaiserslautern,NaN,DE,49.4123,7.69879,NaN,NaN,Paul Walter,Baier
4,4,4305013-1,Kaiserslautern,NaN,DE,49.4123,7.69879,NaN,NaN,Paul Walter,Baier


In [5]:
inventor = inventor[inventor['country'].isin(['US'])]
inventor = inventor.drop_duplicates()

In [6]:
inventor.head(50)

,Unnamed: 0,inventor_id,city,state,country,latitude,longitude,state_fips,county_fips,name_first,name_last
178,178,4388042-1,Birmingham,MI,US,42.5467,-83.2114,26.0,125.0,Thomas Philip,Weber
179,179,4388042-1,Birmingham,MI,US,42.5467,-83.2114,26.0,125.0,Thomas Philip,Weber
180,180,4388042-1,Birmingham,MI,US,42.5467,-83.2114,26.0,125.0,Thomas Philip,Weber
181,181,4388042-1,Birmingham,MI,US,42.5467,-83.2114,26.0,125.0,Thomas Philip,Weber
191,191,4388042-1,Ossining,NY,US,41.1628,-73.8619,36.0,119.0,Thomas Philip,Weber
884,884,5857182-6,San Jose,CA,US,37.3382,-121.8860,6.0,85.0,Serge Philippe,Rielau
993,993,5022698-1,Plymouth,MI,US,42.3714,-83.4703,26.0,163.0,Timothy R.,Butt
994,994,5022698-1,Plymouth,MI,US,42.3714,-83.4703,26.0,163.0,Timothy R.,Butt
995,995,5022698-1,Plymouth,MI,US,42.3714,-83.4703,26.0,163.0,Timothy R.,Butt
996,996,5022698-1,McCordsville,IN,US,39.9081,-85.9228,18.0,59.0,Timothy R.,Butt


In [7]:
inventor['name_last']=clean(inventor['name_last'], lowercase=False, remove_brackets=False)
inventor['name_first']=clean(inventor['name_first'], lowercase=False, remove_brackets=False)
inventor.head()

,Unnamed: 0,inventor_id,city,state,country,latitude,longitude,state_fips,county_fips,name_first,name_last
178,178,4388042-1,Birmingham,MI,US,42.5467,-83.2114,26.0,125.0,Thomas Philip,Weber
179,179,4388042-1,Birmingham,MI,US,42.5467,-83.2114,26.0,125.0,Thomas Philip,Weber
180,180,4388042-1,Birmingham,MI,US,42.5467,-83.2114,26.0,125.0,Thomas Philip,Weber
181,181,4388042-1,Birmingham,MI,US,42.5467,-83.2114,26.0,125.0,Thomas Philip,Weber
191,191,4388042-1,Ossining,NY,US,41.1628,-73.8619,36.0,119.0,Thomas Philip,Weber


## Questsion 2

In [8]:
grants=pd.read_csv("~/projects/wagner_ada_2019/shared/Data/projects/FedRePORTER_PRJ_C_FY2017.csv",error_bad_lines=False, encoding='utf8')

In [9]:
grants.shape

(85343, 24)

In [10]:
grants.head()

,PROJECT_ID,PROJECT_TERMS,PROJECT_TITLE,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,PROJECT_START_DATE,PROJECT_END_DATE,CONTACT_PI_PROJECT_LEADER,...,ORGANIZATION_CITY,ORGANIZATION_STATE,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS
0,919000,Abdomen; abdominal pressure; Acute Renal Fail...,A NOVEL DEVICE FOR OPTIMAL AND PERSONALIZED FL...,HHS,NIH,NIGMS,1R44GM123799-01,9/1/2017,8/31/2019,"BURNETT, DANIEL ROGERS",...,SAN FRANCISCO,CA,941072523,UNITED STATES,9/1/2017,8/31/2018,859.0,2017,1685221.0,NaN
1,919001,Address; Administrative Supplement; African A...,FLORIDA INTERNATIONAL UNIVERSITY-HEALTH DISPAR...,HHS,NIH,NIMHD,5S21MD010683-02,4/1/2016,3/31/2021,"GIL, ANDRES G",...,MIAMI,FL,331990001,UNITED STATES,4/1/2017,3/31/2018,307.0,2017,1900000.0,NaN
2,919002,affiliative behavior; Amygdaloid structure; A...,CHARACTERIZING THE (EPI)GENETICS OF OXYTOCIN R...,HHS,NIH,NICHD,1R01HD088007-01A1,2/14/2017,1/31/2022,"GREGORY, SIMON G",...,DURHAM,NC,277054673,UNITED STATES,2/14/2017,1/31/2018,865.0,2017,656021.0,NaN
3,919003,Acute; Acute Kidney Failure; Adverse event; a...,DEVELOPMENT OF FIBRIN-SPECIFIC NUCLEAR PROBE T...,HHS,NIH,NHLBI,1R42HL139342-01,9/1/2017,12/31/2017,"BLACKLEDGE, JAMES",...,SAINT LOUIS,MO,631464809,UNITED STATES,9/1/2017,12/31/2017,837.0,2017,151903.0,NaN
4,919004,Chemicals; Clinical Research; clinical resear...,CCTN - CONTRACEPTIVE CLINICAL TRIALS NETWORK -...,HHS,NIH,NICHD,275201300017I-7-27500001-1,6/26/2013,6/25/2018,"BORGATTA, LYNN",...,BOSTON,MA,021182908,UNITED STATES,NaN,NaN,NaN,2017,9413.0,NaN


In [11]:
grants.dtypes

PROJECT_ID                       int64
 PROJECT_TERMS                  object
 PROJECT_TITLE                  object
 DEPARTMENT                     object
 AGENCY                         object
 IC_CENTER                      object
 PROJECT_NUMBER                 object
 PROJECT_START_DATE             object
 PROJECT_END_DATE               object
 CONTACT_PI_PROJECT_LEADER      object
 OTHER_PIS                      object
 CONGRESSIONAL_DISTRICT        float64
 DUNS_NUMBER                   float64
 ORGANIZATION_NAME              object
 ORGANIZATION_CITY              object
 ORGANIZATION_STATE             object
 ORGANIZATION_ZIP               object
 ORGANIZATION_COUNTRY           object
 BUDGET_START_DATE              object
 BUDGET_END_DATE                object
 CFDA_CODE                     float64
 FY                              int64
 FY_TOTAL_COST                 float64
 FY_TOTAL_COST_SUB_PROJECTS    float64
dtype: object

In [12]:
grants.rename(columns=lambda x: x.strip(), inplace=True)

In [13]:
grants=(grants[['CONTACT_PI_PROJECT_LEADER', 'OTHER_PIS', 'ORGANIZATION_ZIP','ORGANIZATION_STATE', 'ORGANIZATION_CITY']])

In [14]:
grants.head()

,CONTACT_PI_PROJECT_LEADER,OTHER_PIS,ORGANIZATION_ZIP,ORGANIZATION_STATE,ORGANIZATION_CITY
0,"BURNETT, DANIEL ROGERS","CONNOR, MICHAEL JOSEPH ; JUNCOS, LUIS A. ;",941072523,CA,SAN FRANCISCO
1,"GIL, ANDRES G","DE LA ROSA, MARIO R. ;",331990001,FL,MIAMI
2,"GREGORY, SIMON G","JIANG, YONG-HUI ; MOY, SHERYL S ;",277054673,NC,DURHAM
3,"BLACKLEDGE, JAMES","LANZA, GREGORY M ;",631464809,MO,SAINT LOUIS
4,"BORGATTA, LYNN",NaN,021182908,MA,BOSTON


## Question 3

In [15]:
new=grants['CONTACT_PI_PROJECT_LEADER'].str.split(',', n=1, expand= True)
grants['NAME_LAST']=new[0]
grants["NAME_FIRST"]=new[1]

In [16]:
grants['NAME_LAST']=clean(grants['NAME_LAST'], lowercase=True, remove_brackets=False)

In [17]:
grants['NAME_FIRST']=clean(grants['NAME_FIRST'], lowercase=True, remove_brackets=False)

In [18]:
grants['NAME_LAST']=grants.NAME_LAST.str.capitalize()

In [19]:
grants['NAME_FIRST']=grants.NAME_FIRST.str.title()

In [20]:
grants.head()

,CONTACT_PI_PROJECT_LEADER,OTHER_PIS,ORGANIZATION_ZIP,ORGANIZATION_STATE,ORGANIZATION_CITY,NAME_LAST,NAME_FIRST
0,"BURNETT, DANIEL ROGERS","CONNOR, MICHAEL JOSEPH ; JUNCOS, LUIS A. ;",941072523,CA,SAN FRANCISCO,Burnett,Daniel Rogers
1,"GIL, ANDRES G","DE LA ROSA, MARIO R. ;",331990001,FL,MIAMI,Gil,Andres G
2,"GREGORY, SIMON G","JIANG, YONG-HUI ; MOY, SHERYL S ;",277054673,NC,DURHAM,Gregory,Simon G
3,"BLACKLEDGE, JAMES","LANZA, GREGORY M ;",631464809,MO,SAINT LOUIS,Blackledge,James
4,"BORGATTA, LYNN",NaN,021182908,MA,BOSTON,Borgatta,Lynn


## Question 4

In [21]:
grants['ORGANIZATION_CITY']=clean(grants['ORGANIZATION_CITY'], lowercase=True, remove_brackets=False)

In [22]:
grants['ORGANIZATION_CITY']=grants.ORGANIZATION_CITY.str.title()

In [25]:
grants=grants[grants['ORGANIZATION_STATE'].isin(['AK', 'AL', 'AR', 'AS', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'GU', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MP', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UM', 'UT', 'VA', 'VI', 'VT', 'WA', 'WI', 'WV', 'WY'])]

In [26]:
grants['ORGANIZATION_STATE'].value_counts()

CA    11453
NY     7256
MA     7209
PA     5130
TX     4172
NC     3370
MD     3300
IL     3056
OH     2503
WA     2396
MI     2368
FL     1991
GA     1893
CT     1667
CO     1623
MO     1585
TN     1584
MN     1560
WI     1381
VA     1351
IN     1142
OR     1063
AL     1005
NJ      967
AZ      882
SC      776
RI      762
UT      746
IA      745
DC      689
KY      687
LA      513
KS      456
NE      442
NH      442
NM      423
OK      387
ME      275
AR      213
HI      198
DE      192
MS      182
VT      178
MT      166
NV      143
PR      134
WV      123
AK      103
SD      100
ID       98
ND       81
WY       59
GU       14
VI        4
AS        1
Name: ORGANIZATION_STATE, dtype: int64

## Question 5

In [27]:
import re

In [28]:
grants['ORGANIZATION_ZIP'] = grants['ORGANIZATION_ZIP'].str.extract('(.....)', expand=True)

In [29]:
grants.head()

,CONTACT_PI_PROJECT_LEADER,OTHER_PIS,ORGANIZATION_ZIP,ORGANIZATION_STATE,ORGANIZATION_CITY,NAME_LAST,NAME_FIRST
0,"BURNETT, DANIEL ROGERS","CONNOR, MICHAEL JOSEPH ; JUNCOS, LUIS A. ;",94107,CA,San Francisco,Burnett,Daniel Rogers
1,"GIL, ANDRES G","DE LA ROSA, MARIO R. ;",33199,FL,Miami,Gil,Andres G
2,"GREGORY, SIMON G","JIANG, YONG-HUI ; MOY, SHERYL S ;",27705,NC,Durham,Gregory,Simon G
3,"BLACKLEDGE, JAMES","LANZA, GREGORY M ;",63146,MO,Saint Louis,Blackledge,James
4,"BORGATTA, LYNN",NaN,02118,MA,Boston,Borgatta,Lynn


## Question 6

In [30]:
grants.columns=grants.columns.str.lower()

In [31]:
grants.head()

,contact_pi_project_leader,other_pis,organization_zip,organization_state,organization_city,name_last,name_first
0,"BURNETT, DANIEL ROGERS","CONNOR, MICHAEL JOSEPH ; JUNCOS, LUIS A. ;",94107,CA,San Francisco,Burnett,Daniel Rogers
1,"GIL, ANDRES G","DE LA ROSA, MARIO R. ;",33199,FL,Miami,Gil,Andres G
2,"GREGORY, SIMON G","JIANG, YONG-HUI ; MOY, SHERYL S ;",27705,NC,Durham,Gregory,Simon G
3,"BLACKLEDGE, JAMES","LANZA, GREGORY M ;",63146,MO,Saint Louis,Blackledge,James
4,"BORGATTA, LYNN",NaN,02118,MA,Boston,Borgatta,Lynn


In [32]:
del grants['contact_pi_project_leader']
del grants['other_pis']

In [33]:
grants.head()

,organization_zip,organization_state,organization_city,name_last,name_first
0,94107,CA,San Francisco,Burnett,Daniel Rogers
1,33199,FL,Miami,Gil,Andres G
2,27705,NC,Durham,Gregory,Simon G
3,63146,MO,Saint Louis,Blackledge,James
4,02118,MA,Boston,Borgatta,Lynn


In [34]:
%pwd

u'/projects/wagner_ada_2019/user/sx368'

In [35]:
grants.to_csv(r"~/grants.csv", sep='\t',encoding='utf8')

## Extra Credit

In [36]:
grants2=pd.read_csv("~/projects/wagner_ada_2019/shared/Data/projects/FedRePORTER_PRJ_C_FY2017.csv",error_bad_lines=False, encoding='utf8')

In [37]:
grants2.rename(columns=lambda x: x.strip(), inplace=True)

In [38]:
grants2=(grants2[['ORGANIZATION_NAME']])

In [39]:
grants2['ORGANIZATION_NAME']=grants2.ORGANIZATION_NAME.str.title()

In [40]:
grants2.head()

,ORGANIZATION_NAME
0,Potrero Medical Inc
1,Florida International University
2,Duke University
3,Capella Imaging Llc
4,Boston Medical Center
